# Ludwig Maximilians Universitaet Muenchen Master Thesis

## Study Population


###### Section Objective: Achieve a Data Frame for the study population that contains: the Patient ID (PID), the date of first operation (First_OP), whether there was a revision (1_Revision), the date of the revision (Revision_Datum), the begin of the observation period which is set to 2 year (Begin_BasePeriod) and the end of the observation period (End_BasePeriod). Additionally, for the LSTM Dataframe multiple lines per patient are available, as the observation period is split in subperiods, each with the begin (PeriodStart) and end (PeriodEnd) of subperiod and a subperiod identifier (PeriodNumber). Finally, the sex of the patient is tagged and the age of the patient at the moment of the start of the subperiod is also tagged.

In [ ]:
#Library import
                                                     
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import sys
sys.path.insert(0, '../base_code/Config_Files')
sys.path.insert(0, '../base_code/classes')
import importlib
from Config_V2 import *
from Config_V3 import *
import Config_V2
import Config_V3
from PatientTimeProfiles import *
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeRegressor
import shap
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_curve, auc, confusion_matrix, roc_auc_score
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
from skopt import BayesSearchCV
from sklearn.preprocessing import StandardScaler
import keras_tuner as kt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from keras_tuner.tuners import BayesianOptimization
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.regularizers import l2
from tensorflow.keras.layers import Conv1D
from sklearn.utils import class_weight
import random
import optuna

In [ ]:
#Seed setting
def set_seed(seed_value=42):
    """Fixes random seed for reproducibility."""
    random.seed(seed_value)
    np.random.seed(seed_value)
    tf.random.set_seed(seed_value)

In [ ]:
#Data reading into StudyPopulation
StudyPopulation = pd.read_csv(data_path + new_version_path + "StudyPopulation_v24_08.csv")


In [ ]:
#Begin and End Base Period columns creation
StudyPopulation['First_OP'] = pd.to_datetime(StudyPopulation['First_OP'], format='%Y-%m-%d')
StudyPopulation['Begin_BasePeriod'] = StudyPopulation['First_OP'] + timedelta(days=1)
StudyPopulation['End_BasePeriod'] = StudyPopulation['First_OP'] + timedelta(days=ObservationDays)

In [ ]:
#Function to create DataFrame with multiple lines per PID breaking the Base Period in months or quarters

def create_timeframe_StudyPopulation(StudyPopulation, TimeResolution):
    # Determine the increment days based on TimeResolution
    match TimeResolution:
        case "Months": 
            increment_days =  29
        case "Quarters": 
            increment_days = 89

    #Fixes the last date where data is available, so subperiods stop being generated where there is no data
    max_end_date = pd.Timestamp(MaxInfoDate)
    adjusted_end_dates = StudyPopulation['End_BasePeriod'].apply(lambda x: min(x, max_end_date))


    # Calculate the total number of periods each original row will be split into. Final plus 1 is since the use of the floor division and that
    # the difference between Begin and End Base Period has 1 day less than exact
    num_periods = ((adjusted_end_dates - StudyPopulation['Begin_BasePeriod']).dt.days // (increment_days + 1)) + 1
    
    # Create arrays for PeriodStart and PeriodEnd
    period_starts = []
    period_ends = []
    row_ids = []
    period_numbers = []

    #zip creates lines each with an element from each involved column. Takes a begin and end from a base period and runs for as many times as num 
    #periods there are, setting the start and end of subperiods, as well as the count of subperiods created
    for i, (start_date, end_date, periods) in enumerate(zip(StudyPopulation['Begin_BasePeriod'], adjusted_end_dates, num_periods)):
        for j in range(periods):
            current_start = start_date + pd.Timedelta(days=j * (increment_days + 1))
            current_end = min(current_start + pd.Timedelta(days=increment_days), end_date)
            
            #Saves start and end and number of subperiod
            period_starts.append(current_start)
            period_ends.append(current_end)
            row_ids.append(i)
            period_numbers.append(j + 1)

    # Create the expanded DataFrame
    expanded_df = pd.DataFrame({
        'PeriodStart': period_starts,
        'PeriodEnd': period_ends,
        'RowID': row_ids,
        'PeriodNumber': period_numbers
    })

    # Merge with the original StudyPopulation DataFrame on the RowID and then drops
    IStudyPopulation = expanded_df.merge(StudyPopulation, left_on='RowID', right_index=True).drop(columns=['RowID'])

    return IStudyPopulation

In [ ]:
#Calling of the function providing a particular DataFrame and time frame
#date of revision and first operation date missing
IStudyPopulation = create_timeframe_StudyPopulation(StudyPopulation, TimeResolution)
IStudyPopulation = IStudyPopulation[["PID","1_Revision","PeriodNumber", "PeriodStart", "PeriodEnd"]]

In [ ]:
increment_days

## Age and Sex

In [ ]:
#Age and Sex Data reading
AgeAndSex = pd.read_csv(data_path + new_version_path + "VERS_STAMM.csv", sep=";")
AgeAndSex = AgeAndSex[["PID","Geburtsdatum","Geschlecht"]]
AgeAndSex = AgeAndSex.drop_duplicates(subset=['PID'])
AgeAndSex

In [ ]:
#Prepare the Age and Sex columns
# Step 1: Extract the year from 'Geburtsdatum'
AgeAndSex['Geburtsdatum'] = pd.to_datetime(AgeAndSex['Geburtsdatum'], format='%d%b%Y').dt.year

# Step 2: Merge AgeAndSex with IStudyPopulation on PID
IStudyPopulationAS = IStudyPopulation.merge(AgeAndSex[['PID', 'Geburtsdatum', 'Geschlecht']], on='PID', how='left')

# Step 3: Calculate Age as the difference between the year of 'PeriodStart' and 'YearOfBirth'
IStudyPopulationAS['PeriodStart'] = pd.to_datetime(IStudyPopulationAS['PeriodStart'], format='%Y-%m-%d')
IStudyPopulationAS['Age'] = IStudyPopulationAS['PeriodStart'].dt.year - IStudyPopulationAS['Geburtsdatum']

# Rename Geschlecht to Sex and drop YearOfBirth if not needed
IStudyPopulationAS.rename(columns={'Geschlecht': 'Sex'}, inplace=True)
IStudyPopulationAS.drop(columns=['Geburtsdatum'], inplace=True)

# Display the final DataFrame
print(IStudyPopulationAS)

## Profiling

###### Section Objective: A DataFrame is built to achieve a matrix structure where rows represent (PID, subperiod number after operation), columns represent for example, ATC_Codes (Medicine code) and 1s are stock whenever there is an existing medicine assigned to the patient in the subperiod, 0s shall be in the full matrix whenever it was not assigned during the subperiod for him/her. The same is done for Treatments and for Diagnostics

## Prescription part

In [ ]:
#Data reading into and truncates ATC_Codes
#PZN means "Pharmazentralnummer" or Central number of medicine
#FG means "Faktorgruppe oder Fachgruppe"
Prescriptions = pd.read_csv(data_path + new_version_path + "AM_EVO.csv", sep=";")
Prescriptions["ATCX"] = Prescriptions["ATC_Code"].str[:ATCX]
Prescriptions

In [ ]:
#Stay just with the desired columns
PrescriptionsF = Prescriptions[["PID","Verordnungsdatum","Anzahl_Packungen","ATCX"]]#"ATC_Presence"]]
PrescriptionsF

In [ ]:
#CREATING A DATAFRAME WITH ATCX AND MENGE AND DDD_JE_PACKUNG FOR EACH ATCX VALUE, SO VALUES CAN BE BROUGHT FOR QUANTITY OF SUBSTANCES CALCULATION

some_df = pd.read_csv(data_path_DZO + new_version_path + "AM_EVO_DDD.csv", delimiter= ";")

# Step 1: Keep only the relevant variables in the DataFrame
some_df = some_df[["ATC_Code", "MENGE", "DDD_JE_PACKUNG"]]

# Step 2: Create the ATCX column by taking the first `ATCX` characters from ATC_Code
# Assuming `ATCX` is defined in a config file as an integer
some_df["ATCX"] = some_df["ATC_Code"].str[:ATCX]

# Step 3: Drop the ATC_Code column
some_df = some_df.drop(columns=["ATC_Code"])

# Step 4: Keep only one row per unique ATCX value, along with the first MENGE and DDD_JE_PACKUNG
some_df = some_df.groupby("ATCX", as_index=False).first()

In [ ]:
PrescriptionsF = pd.merge(PrescriptionsF, some_df, on="ATCX", how="left")

# Display the resulting DataFrame
print(PrescriptionsF.head())

## Treatment part

In [ ]:
Treatments = pd.read_csv(data_path + new_version_path + "HEMI_EVO.csv", sep=";")
Treatments = Treatments[["PID", "Leistung","Leistungsdatum"]]
Treatments 

## Diagnostics part

In [ ]:
Diagnostics = pd.read_csv(data_path + new_version_path + "ARZT_DIAGNOSE.csv", sep=";")
Diagnostics = Diagnostics[["PID", "Bezugsjahr", "ICD_Code", "Behandl_Quartal"]]
Diagnostics["ICD_Code"] = Diagnostics["ICD_Code"].str[:AICDX]

# Mapping quarter numbers to month abbreviations
quarter_to_month = {1: 'JAN', 2: 'APR', 3: 'JUL', 4: 'OCT'}

# Create the new date column with format '01MONYEAR'
Diagnostics['Bezugsjahr'] = '01' + Diagnostics['Behandl_Quartal'].map(quarter_to_month) + Diagnostics['Bezugsjahr'].astype(str)

# Drop the Behandl_Quartal column as requested
Diagnostics = Diagnostics.drop(columns=['Behandl_Quartal'])

Diagnostics

## Profile building

### Distribution of Days between the first operation date and the the first treatment date 

In [ ]:
# DATAFRAME FUNCTION CALLING: 
profiles = PatientTimeProfiles(IStudyPopulationAS, PrescriptionsF, Treatments, Diagnostics)
result_df = profiles.create_sparse_dataframe()
print(result_df.head()) 


#IF RUNNING profiles.create_sparse_dataframe_first_30_days instead of create_sparse_dataframe FOR CORRELATION BETWEEN PAIN KILLER QUANTITY AND HEMI_GAP
#RUN THIS FOLLOWING CELL, LEAVE ONE AND RUN THE NEXT, AND THEN DIRECTLY UNTIL CORRELATION CODE

In [ ]:
### Distribution of Days between the first operation date and the the first treatment date 
# HYPOTHESIS: PRESCRIPTIONS AND TREATMENT CLOSE AFTER THEIR OPERATION RECOVER BETTER

# Step 1: Filter Treatments to keep only PIDs that are in Study Population  reatments['PID'].isin(StudyPopulation['PID'])]
filtered_treatments = Treatments[Treatments['PID'].isin(StudyPopulation['PID'])]


# Step 2: Merge StudyPopulation with filtered treatments to have access to First_OP during filtering
merged_treatments = pd.merge(filtered_treatments, StudyPopulation[['PID', 'First_OP']], on='PID', how='left')

# Step 3: Filter treatments to keep only those where Leistungsdatum is after First_OP
valid_treatments = merged_treatments[merged_treatments['Leistungsdatum'] > merged_treatments['First_OP']]

# Step 4: Group by PID and get the earliest Leistungsdatum after the First_OP, calculate difference in days and replace all ones larger than observed years with 0
earliest_treatments_after_op = valid_treatments.groupby('PID').agg({'Leistungsdatum': 'min', 'Leistung': 'first'}).reset_index()
earliest_treatments_after_op = pd.merge(earliest_treatments_after_op, StudyPopulation[['PID', 'First_OP']], on='PID', how='left')
earliest_treatments_after_op['days_diff'] = (earliest_treatments_after_op['Leistungsdatum'] - earliest_treatments_after_op['First_OP']).dt.days
earliest_treatments_after_op.loc[earliest_treatments_after_op['days_diff'] > ObservationDays, ['Leistungsdatum', 'Leistung']] = np.nan

#HYPOTHESIS: PEOPLE TAKING P
# Step 5: Merge the earliest valid treatment with Study Population
merged_df = pd.merge(StudyPopulation, earliest_treatments_after_op[["PID", "Leistungsdatum", "Leistung"]], on='PID', how='left')

merged_df['First_OP'] = pd.to_datetime(merged_df['First_OP'])
merged_df['Leistungsdatum'] = pd.to_datetime(merged_df['Leistungsdatum'])

# Step 6: Calculate the difference in days
merged_df['Treatment_Gap_HEMI'] = (merged_df['Leistungsdatum'] - merged_df['First_OP']).dt.days

# Step 7: Plot the distribution of the number of days
plt.figure(figsize=(10, 6))
plt.hist(merged_df['Treatment_Gap_HEMI'], bins=30, edgecolor='black')
plt.xlabel('Difference in Days')
plt.ylabel('Number of Cases')
plt.title('Distribution of Days Between First_OP and First Treatment')
plt.show()

# Output the final DataFrame with the calculated differences
print(merged_df)


#QUESTIONS: 

#Here, a separation for the population that received a revision and the one which would not would also be good, 
#right? Hypothesis: In the ones which did not receive revision, there shall be more cases which received treatment
#quickly. A: YUP 

### Division between Revisioned and non Revisioned populations

In [ ]:
#New code: First creates an array of the same shape, 1 row for each PID with a 1 instead of the number of days, and divides each 1 over the total 
#number of PID in that subpopulation, therefore, the percentage each PID carries for the full population. Finally for each distinct value, it adds
#the percentual contribution of each time it finds a such case.

merged_df_rev = merged_df[merged_df["1_Revision"] == 1]
merged_df_no_rev = merged_df[merged_df["1_Revision"] == 0]

# Calculate the relative percentage for each population
rev_weights = np.ones_like(merged_df_rev['Treatment_Gap_HEMI']) / len(merged_df_rev)
no_rev_weights = np.ones_like(merged_df_no_rev['Treatment_Gap_HEMI']) / len(merged_df_no_rev)

# Plot both histograms in the same figure
plt.figure(figsize=(10, 6))
plt.hist(merged_df_rev['Treatment_Gap_HEMI'], bins=30, edgecolor='black', alpha=0.7, label='With Revision', color='blue', weights=rev_weights)
plt.hist(merged_df_no_rev['Treatment_Gap_HEMI'], bins=30, edgecolor='black', alpha=0.7, label='Without Revision', color='orange', weights=no_rev_weights)

# Add labels and title
plt.xlabel('Difference in Days')
plt.ylabel('Percentage of Cases')
plt.title('Distribution of Days Between First_OP and First Treatment')
plt.legend()

# Show the plot
plt.show()


In [ ]:
#Add the Treatment_Gap_HEMI variable to the result_df Dataset
result_df = pd.merge(result_df, merged_df[['PID', 'Treatment_Gap_HEMI']], on='PID', how='left')

### Analysis for Prescriptions

In [ ]:
# Step 1: Filter Prescriptions to keep only PIDs that are in Study Population
filtered_prescriptions = PrescriptionsF[PrescriptionsF['PID'].isin(StudyPopulation['PID'])]

# Step 2: Merge StudyPopulation with filtered prescriptions to have access to First_OP during filtering
merged_prescriptions = pd.merge(filtered_prescriptions, StudyPopulation[['PID', 'First_OP']], on='PID', how='left')

# Step 3: Filter prescriptions to keep only those where Verordnungsdatum is after First_OP
valid_prescriptions = merged_prescriptions[merged_prescriptions['Verordnungsdatum'] > merged_prescriptions['First_OP']]

# Step 4: Group by PID and get the earliest Verordnungsdatum after the First_OP
earliest_prescriptions_after_op = valid_prescriptions.groupby('PID').agg({'Verordnungsdatum': 'min', 'ATCX': 'first'}).reset_index()
earliest_prescriptions_after_op = pd.merge(earliest_prescriptions_after_op, StudyPopulation[['PID', 'First_OP']], on='PID', how='left')
earliest_prescriptions_after_op['days_diff'] = (earliest_prescriptions_after_op['Verordnungsdatum'] - earliest_prescriptions_after_op['First_OP']).dt.days
earliest_prescriptions_after_op.loc[earliest_prescriptions_after_op['days_diff'] > ObservationDays, ['Verordnungsdatum', 'ATCX']] = np.nan

# Step 5: Merge the earliest valid treatment with Study Population
merged_df = pd.merge(StudyPopulation, earliest_prescriptions_after_op[["PID", "Verordnungsdatum", "ATCX"]], on='PID', how='left')

merged_df['First_OP'] = pd.to_datetime(merged_df['First_OP'])
merged_df['Verordnungsdatum'] = pd.to_datetime(merged_df['Verordnungsdatum'])

# Step 6: Calculate the difference in days
merged_df['Treatment_Gap_ATCX'] = (merged_df['Verordnungsdatum'] - merged_df['First_OP']).dt.days

# Step 7: Plot the distribution of the number of days
plt.figure(figsize=(10, 6))
plt.hist(merged_df['Treatment_Gap_ATCX'], bins=30, edgecolor='black')
plt.xlabel('Difference in Days')
plt.ylabel('Number of Cases')
plt.title('Distribution of Days Between First_OP and First Treatment')
plt.show()

# Output the final DataFrame with the calculated differences
print(merged_df)

In [ ]:
#Division between population with and without Revision

merged_df_rev = merged_df[merged_df["1_Revision"] == 1]
merged_df_no_rev = merged_df[merged_df["1_Revision"] == 0]
 
# Calculate the relative percentage for each population
rev_weights = np.ones_like(merged_df_rev['Treatment_Gap_ATCX']) / len(merged_df_rev)
no_rev_weights = np.ones_like(merged_df_no_rev['Treatment_Gap_ATCX']) / len(merged_df_no_rev)

# Plot both histograms in the same figure
plt.figure(figsize=(10, 6))
plt.hist(merged_df_rev['Treatment_Gap_ATCX'], bins=30, edgecolor='black', alpha=0.7, label='With Revision', color='blue', weights=rev_weights)
plt.hist(merged_df_no_rev['Treatment_Gap_ATCX'], bins=30, edgecolor='black', alpha=0.7, label='Without Revision', color='orange', weights=no_rev_weights)

# Add labels and title
plt.xlabel('Difference in Days')
plt.ylabel('Percentage of Cases')
plt.title('Distribution of Days Between First_OP and First Prescription')
plt.legend()

In [ ]:
#Adding of the Treatment_Gap_ATCX variable to result_df
result_df = pd.merge(result_df, merged_df[['PID', 'Treatment_Gap_ATCX']], on='PID', how='left')

## Separation of population between Pain Killers and Non Pain Killers

In [ ]:

# Step 1: Filter Prescriptions to keep only PIDs that are in Study Population
filtered_prescriptions = PrescriptionsF[PrescriptionsF['PID'].isin(StudyPopulation['PID'])]

# Step 2: Merge StudyPopulation with filtered prescriptions to have access to First_OP during filtering
merged_prescriptions = pd.merge(filtered_prescriptions, StudyPopulation[['PID', 'First_OP']], on='PID', how='left')

# Step 3: Filter prescriptions to keep only those where Verordnungsdatum is after First_OP
valid_prescriptions = merged_prescriptions[merged_prescriptions['Verordnungsdatum'] > merged_prescriptions['First_OP']]

# Step 3.5: Split the data based on ATCX code starting with 'N02' and not
prescriptions_n02 = valid_prescriptions[valid_prescriptions['ATCX'].str.startswith('N02')]
prescriptions_other = valid_prescriptions[~valid_prescriptions['ATCX'].str.startswith('N02')]

# Step 4: Process each group separately to get the earliest Verordnungsdatum after First_OP for each PID
# Group for prescriptions with ATCX codes starting with 'N02'
earliest_n02_after_op = prescriptions_n02.groupby('PID').agg({'Verordnungsdatum': 'min', 'ATCX': 'first'}).reset_index()
earliest_n02_after_op = pd.merge(earliest_n02_after_op, StudyPopulation[['PID', 'First_OP']], on='PID', how='left')
earliest_n02_after_op['days_diff'] = (earliest_n02_after_op['Verordnungsdatum'] - earliest_n02_after_op['First_OP']).dt.days
earliest_n02_after_op.loc[earliest_n02_after_op['days_diff'] > ObservationDays, ['Verordnungsdatum', 'ATCX']] = np.nan

# Group for prescriptions with ATCX codes not starting with 'N02'
earliest_other_after_op = prescriptions_other.groupby('PID').agg({'Verordnungsdatum': 'min', 'ATCX': 'first'}).reset_index()
earliest_other_after_op = pd.merge(earliest_other_after_op, StudyPopulation[['PID', 'First_OP']], on='PID', how='left')
earliest_other_after_op['days_diff'] = (earliest_other_after_op['Verordnungsdatum'] - earliest_other_after_op['First_OP']).dt.days
earliest_other_after_op.loc[earliest_other_after_op['days_diff'] > ObservationDays, ['Verordnungsdatum', 'ATCX']] = np.nan

# Step 5: Merge each group's earliest valid treatment with the Study Population
merged_n02 = pd.merge(StudyPopulation, earliest_n02_after_op[["PID", "Verordnungsdatum", "ATCX"]], on='PID', how='left')
merged_other = pd.merge(StudyPopulation, earliest_other_after_op[["PID", "Verordnungsdatum", "ATCX"]], on='PID', how='left')

for df in [merged_n02, merged_other]:
    df['First_OP'] = pd.to_datetime(df['First_OP'])
    df['Verordnungsdatum'] = pd.to_datetime(df['Verordnungsdatum'])
    df['Treatment_Gap_ATCX'] = (df['Verordnungsdatum'] - df['First_OP']).dt.days

# Optional: Plot or analyze each group individually, for example:
plt.figure(figsize=(10, 6))
plt.hist(merged_n02['Treatment_Gap_ATCX'].dropna(), bins=30, edgecolor='black', alpha=0.7, label='ATCX starts with N02')
plt.hist(merged_other['Treatment_Gap_ATCX'].dropna(), bins=30, edgecolor='black', alpha=0.7, label='ATCX does not start with N02')
plt.xlabel('Difference in Days')
plt.ylabel('Number of Cases')
plt.title('Distribution of Days Between First_OP and First Prescription')
plt.legend()
plt.show()

In [ ]:

#Division between population with and without Revision

merged_n02_rev = merged_n02[merged_n02["1_Revision"] == 1]
merged_n02_no_rev = merged_n02[merged_n02["1_Revision"] == 0]
 
# Calculate the relative percentage for each population
rev_weights = np.ones_like(merged_n02_rev['Treatment_Gap_ATCX']) / len(merged_n02_rev)
no_rev_weights = np.ones_like(merged_n02_no_rev['Treatment_Gap_ATCX']) / len(merged_n02_no_rev)

# Plot both histograms in the same figure
plt.figure(figsize=(10, 6))
plt.hist(merged_n02_rev['Treatment_Gap_ATCX'], bins=30, edgecolor='black', alpha=0.7, label='With Revision', color='blue', weights=rev_weights)
plt.hist(merged_n02_no_rev['Treatment_Gap_ATCX'], bins=30, edgecolor='black', alpha=0.7, label='Without Revision', color='orange', weights=no_rev_weights)

# Add labels and title
plt.xlabel('Difference in Days')
plt.ylabel('Percentage of Cases')
plt.title('Distribution of Days Between First_OP and First Prescription')
plt.legend()

In [ ]:
#Therefore, this story tells that people who got a revision, were probably under more pain in the moment of first operation, and therefore started to
#take their pain killers before, than those who didn´t get a revision, which were also not in so much pain and in proportion didn´t start taking 
#pain killers in the same way.

#HYPOTHESIS: Based on field knowledge, we believe that the larger the number of pain killers a patient takes, the more it takes them to start to 
#their therapies, and the other way around, patients without so many pain killer drugs assigned begin with their therapies before. Therefore: 
# we´ll measure the correlation between pain killer amount and number of days to first treatment. We also divide patients in groups by their painkiller
#usage level, and see the average Treatment_Gap_HEMI to be compared betweeen groups with a t-test

In [ ]:
#WARNING: RUN JUST WHEN DOING THE CORRELATION ANALYSIS BETWEEN PAIN KILLER USAGE AND HEMI GAP, RUNNING create_sparse_dataframe_first_30_days

from matplotlib.ticker import PercentFormatter

#STEP I
result_df['total_painkillers'] = result_df.filter(regex='^ATCX_N02').sum(axis=1)



#STEP II
# Filtering Treatments and calculating the first treatment date after operation date
filtered_treatments = Treatments[Treatments['PID'].isin(StudyPopulation['PID'])]
merged_treatments = pd.merge(filtered_treatments, StudyPopulation[['PID', 'First_OP']], on='PID', how='left')
valid_treatments = merged_treatments[merged_treatments['Leistungsdatum'] > merged_treatments['First_OP']]

# Getting the earliest treatment after the operation for each patient
earliest_treatments_after_op = valid_treatments.groupby('PID').agg({
    'Leistungsdatum': 'min'
}).reset_index()

# Calculate days difference and handle cases beyond observed period
earliest_treatments_after_op = pd.merge(earliest_treatments_after_op, StudyPopulation[['PID', 'First_OP']], on='PID', how='left')
earliest_treatments_after_op['Treatment_Gap_HEMI'] = (earliest_treatments_after_op['Leistungsdatum'] - earliest_treatments_after_op['First_OP']).dt.days
earliest_treatments_after_op.loc[earliest_treatments_after_op['Treatment_Gap_HEMI'] > ObservationDays, 'Treatment_Gap_HEMI'] = np.nan

#STEP III
# Merge the calculated treatment gap with the patient drug summary
analysis_df = pd.merge(result_df[['PID', 'total_painkillers']], earliest_treatments_after_op[['PID', 'Treatment_Gap_HEMI']], on='PID', how='left')

# Checking correlation between painkiller usage and treatment gap
correlation = analysis_df['total_painkillers'].corr(analysis_df['Treatment_Gap_HEMI'])
print(f"Correlation between painkiller usage and days to first treatment: {correlation}")



#STEP IV
# Define a threshold to categorize patients as high or low painkiller users
painkiller_threshold = analysis_df['total_painkillers'].median()
analysis_df['painkiller_group'] = np.where(analysis_df['total_painkillers'] >= painkiller_threshold, 'High', 'Low')

# Calculate average treatment gap for each group
group_means = analysis_df.groupby('painkiller_group')['Treatment_Gap_HEMI'].mean()
print(group_means)

# Perform t-test or Mann-Whitney U test to test for significant difference
from scipy.stats import ttest_ind, mannwhitneyu

high_group = analysis_df[analysis_df['painkiller_group'] == 'High']['Treatment_Gap_HEMI'].dropna()
low_group = analysis_df[analysis_df['painkiller_group'] == 'Low']['Treatment_Gap_HEMI'].dropna()

t_stat, p_val = ttest_ind(high_group, low_group)
print(f"T-test results: t-statistic={t_stat}, p-value={p_val}")


# STEP IV: Plotting

# Plot 1: Distribution of Treatment Gap by Painkiller Usage Group
plt.figure(figsize=(10, 6))
for group, color in zip(['High', 'Low'], ['blue', 'orange']):
    values = analysis_df[analysis_df['painkiller_group'] == group]['Treatment_Gap_HEMI'].dropna()
    plt.hist(
        values,
        bins=30, alpha=0.7, label=f"{group} Painkiller Usage", color=color,
        weights=np.ones_like(values) / len(values)
    )
plt.xlabel('Days to First Treatment')
plt.ylabel('Percentage of Population')
plt.title('Distribution of Treatment Gap by Painkiller Usage Group')
plt.gca().yaxis.set_major_formatter(PercentFormatter(1))
plt.legend()
plt.show()

# Plot 2: Boxplot of Treatment Gaps
plt.figure(figsize=(8, 6))
analysis_df.boxplot(column='Treatment_Gap_HEMI', by='painkiller_group', grid=False, patch_artist=True)
plt.title('Boxplot of Treatment Gap by Painkiller Usage Group')
plt.suptitle("")  # Suppress the default title
plt.xlabel('Painkiller Usage Group')
plt.ylabel('Days to First Treatment')
plt.show()


#Plot 3: Scatterplot Pain Killer level and Treatment Gap
# Ensure no NaN values in the columns used for plotting
plot_df = analysis_df[['total_painkillers', 'Treatment_Gap_HEMI']].dropna()
plot_df_filtered = plot_df[plot_df['total_painkillers'] <= 20]

# Create the scatter plot
plt.figure(figsize=(10, 6))
sns.scatterplot(data=plot_df_filtered, x='Treatment_Gap_HEMI', y='total_painkillers', alpha=0.7, color='blue')

# Add labels and title
plt.xlabel('Days to First Treatment')
plt.ylabel('Painkiller Volume')
plt.title('Correlation Between Painkiller Usage and Days to First Treatment')

# Show the plot
plt.show()

In [ ]:
#CORRELATION ANALYSIS
#An unexpected low and even negative! correlation between painkiller usage and days to first treatment. In theory we expected, that when high quantities of 
#painkillers would be used, larger days to treatment would follow because of inability to join because of pain. The negative correlation sells the 
#opposite story, either way, the magnitude of the correlation is close to 0.

#After splitting populations of larger than median and lower than median pain killer usage, the average for their HEMI GAPS was calculated and the t test 
#suggests there is no significative difference between the both means, also suggesting pain killer usage does not cause the number of days to 
#first treatment to be larger. 

#First plot even says people in pain tend to start their treatment a little bit before in comparisson to people without so much pain.

#Second plot. Median for Days to First Treatment appears to be similar for both groups with high and low pain killer usage, there is also similar box 
#extension, having similar +-1 quantiles from the median. Both groups with patients with significantly larger waiting times to first treatment. 


### Analysis for Diagnosis


In [ ]:
# Step 1: Filter Prescriptions to keep only PIDs that are in Study Population
filtered_diagnostics = Diagnostics[Diagnostics['PID'].isin(StudyPopulation['PID'])]

# Step 2: Merge StudyPopulation with filtered prescriptions to have access to First_OP during filtering
merged_diagnostics = pd.merge(filtered_diagnostics, StudyPopulation[['PID', 'First_OP']], on='PID', how='left')

# Step 3: Filter prescriptions to keep only those where Verordnungsdatum is after First_OP
valid_diagnostics = merged_diagnostics[merged_diagnostics['Bezugsjahr'] > merged_diagnostics['First_OP']]

# Step 4: Group by PID and get the earliest Verordnungsdatum after the First_OP
earliest_diagnostics_after_op = valid_diagnostics.groupby('PID').agg({'Bezugsjahr': 'min', 'ICD_Code': 'first'}).reset_index()
earliest_diagnostics_after_op = pd.merge(earliest_diagnostics_after_op, StudyPopulation[['PID', 'First_OP']], on='PID', how='left')
earliest_diagnostics_after_op['days_diff'] = (earliest_diagnostics_after_op['Bezugsjahr'] - earliest_diagnostics_after_op['First_OP']).dt.days
earliest_diagnostics_after_op.loc[earliest_diagnostics_after_op['days_diff'] > ObservationDays, ['Bezugsjahr', 'ICD_Code']] = np.nan

# Step 5: Merge the earliest valid treatment with Study Population
merged_df = pd.merge(StudyPopulation, earliest_diagnostics_after_op[["PID", "Bezugsjahr", "ICD_Code"]], on='PID', how='left')

merged_df['First_OP'] = pd.to_datetime(merged_df['First_OP'])
merged_df['Bezugsjahr'] = pd.to_datetime(merged_df['Bezugsjahr'])

# Step 6: Calculate the difference in days
merged_df['Treatment_Gap_AICDX'] = (merged_df['Bezugsjahr'] - merged_df['First_OP']).dt.days

# Step 7: Plot the distribution of the number of days
plt.figure(figsize=(10, 6))
plt.hist(merged_df['Treatment_Gap_AICDX'], bins=30, edgecolor='black')
plt.xlabel('Difference in Days')
plt.ylabel('Number of Cases')
plt.title('Distribution of Days Between First_OP and First Diagnostics')
plt.show()

# Output the final DataFrame with the calculated differences
print(merged_df)

In [ ]:
#Division between population with and without Revision

merged_df_rev = merged_df[merged_df["1_Revision"] == 1]
merged_df_no_rev = merged_df[merged_df["1_Revision"] == 0]
 
# Calculate the relative percentage for each population
rev_weights = np.ones_like(merged_df_rev['Treatment_Gap_AICDX']) / len(merged_df_rev)
no_rev_weights = np.ones_like(merged_df_no_rev['Treatment_Gap_AICDX']) / len(merged_df_no_rev)

# Plot both histograms in the same figure
plt.figure(figsize=(10, 6))
plt.hist(merged_df_rev['Treatment_Gap_AICDX'], bins=30, edgecolor='black', alpha=0.7, label='With Revision', color='blue', weights=rev_weights)
plt.hist(merged_df_no_rev['Treatment_Gap_AICDX'], bins=30, edgecolor='black', alpha=0.7, label='Without Revision', color='orange', weights=no_rev_weights)

# Add labels and title
plt.xlabel('Difference in Days')
plt.ylabel('Percentage of Cases')
plt.title('Distribution of Days Between First_OP and First Diagnostics')
plt.legend()

# Show the plot
plt.show()

In [ ]:
#Adding of the Treatment_Gap_AICDX variable
result_df = pd.merge(result_df, merged_df[['PID', 'Treatment_Gap_AICDX']], on='PID', how='left')

In [ ]:
#RUN THIS EVERY TIME
filtered_df = result_df
#filtered_df

In [ ]:
filtered_df.head()

In [ ]:
#SKIP THIS AND WHOLE FOLLOWING SECTION IF NOT WISHING TO PRODUCE PLOTS FOR DATA SECTION
reduced_result_df = result_df.groupby('PID', as_index=False).first()

## Plot producing section for Data section

In [ ]:
# Count of patients who received and did not receive revision
revision_counts = reduced_result_df['1_Revision'].value_counts()

# Plotting the counts
plt.figure(figsize=(6, 4))
revision_counts.plot(kind='bar', color=['blue', 'orange'], alpha=0.7)

# Add counts above the bars
for idx, value in enumerate(revision_counts):
    plt.text(x=idx, y=value + 0.5, s=str(value), ha='center', fontsize=10)
    
plt.title("Count of Patients: Revision vs. No Revision")
plt.xlabel("Revision Status (0 = No, 1 = Yes)")
plt.ylabel("Count of Patients")
plt.xticks(rotation=0)
plt.show()

In [ ]:
# Calculate statistics
age_stats = reduced_result_df['Age'].describe()
print(f"Age Statistics:\n{age_stats}")


# Boxplot
plt.figure(figsize=(6, 4))
plt.boxplot(reduced_result_df['Age'], vert=False, patch_artist=True, boxprops=dict(facecolor='skyblue', color='black'))
plt.title("Age Boxplot")
plt.xlabel("Age")
plt.show()

In [ ]:
# Count of 1s and 2s
sex_count = reduced_result_df['Sex'].value_counts()

# Plotting
plt.figure(figsize=(6, 4))
sex_count.plot(kind='bar', color=['green', 'red'], alpha=0.7)
plt.title("Count of 1s and 2s in Sex")
plt.xlabel("Sex Group")
plt.ylabel("Count")
plt.xticks(rotation=0)
plt.show()

In [ ]:
# List of treatment gaps
treatment_gaps = ['Treatment_Gap_ATCX', 'Treatment_Gap_HEMI', 'Treatment_Gap_AICDX']

for gap in treatment_gaps:
    # Restore original values if previously transformed
    reduced_result_df[gap] = np.exp(reduced_result_df[gap])  # Uncomment if needed

    print(f"\nStatistics for {gap} (Restored Values):\n{reduced_result_df[gap].describe()}")
   
    # Quantile plot
    plt.figure(figsize=(6, 4))
    reduced_result_df[gap].quantile([0.0, 0.25, 0.5, 0.75, 1.0]).plot(kind='bar', color='purple')
    plt.title(f"Quantiles for {gap}")
    plt.xlabel("Quantiles")
    plt.ylabel("Days")
    plt.show()

In [ ]:
columns_of_interest = ['ATCX_N02A', 'HEMI_0', 'AICDX_F32']

# Count rows with a 1 for each column
subperiod_counts = {}
total_rows = len(result_df)

for column in columns_of_interest:
    count = result_df[column].sum()
    proportion = count / total_rows
    subperiod_counts[column] = {"Count": count, "Proportion": proportion}
    print(f"{column}: {count} rows with 1, {proportion:.2%} of subperiods")

# Convert to DataFrame for visualization
subperiod_df = pd.DataFrame.from_dict(subperiod_counts, orient='index')

# Plot proportions
subperiod_df['Proportion'].plot(kind='bar', color='teal', alpha=0.7, figsize=(6, 4))
plt.title("Proportion of Subperiods with Assigned Variables")
plt.xlabel("Variable")
plt.ylabel("Proportion")
plt.xticks(rotation=45)
plt.show()

### Modeling little research

##### Modeling options: Markov Models: Assumptions: Whether a patient will receive a revision or not depends on the current moment, and not in the sequence of of moments before. The health status of the patient is hidden and only seen through the medication and treatment he receives. Here, hidden states would be health states of the patient. Next state depends only on current one. There is a probability distribution for the assignment of prescriptions and treatments for each hidden state. Finally, whether the patient receives a revision or not is determined by the path of states of the patient. With probabilities to transition from one state to another. Could use the EM algorithm to train the model. You could use your model to predict states of patients given a path, or the probability of needing revision. May create over simple models and the assumptions might not be fulfilled. 

##### LSTM: Capture sequential patterns. There is no such assumption that the probability of needing revision is just influenced by the immediate state before, but are built to capture long term relationships in sequences. The model would predict the probability of needed revision at each time step of the path. Frequencies of prescriptions or dosages and patient characteristics could also be features of our model. Train using binary cross entropy loss. Non linear relationships can be modeled by LSTM. A lot of data is needed to train them. Black boxy. 

##### Posibility: Transformers: They process the entire sequence at a single step. Use attention to compare each element of the sequence to the other elements. They can speed up. They are more transparent than LSTMs. LSTMs need less data, for small datasets, they can overfit easily. Apparently because of the size of the data we are working with, LSTM seems better.



## Preliminary Decision Tree Model, Evaluation metrics and Shapley Analysis

In [ ]:
#Creation of Data for Decision Tree feeding: One line per PID covering the full time '
#PID: Tomar uno, todo igual. Quita subperiod, PeriodStart y PeriodEnd, 1_Revision y Sex el que sea, Age es del 
#primer subperiodo, ATCX HEMI AICDX: Si es 1 en algún subperiodo es 1, else 0. Gaps el que sea.

grouped = filtered_df.groupby('PID')

# Aggregate the data
filtered_df_DT = grouped.agg(
    {
        '1_Revision': 'first',
        'Sex': 'first',
        'Age': 'first',
        'Treatment_Gap_ATCX': 'first',
        'Treatment_Gap_HEMI': 'first',
        'Treatment_Gap_AICDX': 'first',
        # Apply logical OR to ATCX_*, HEMI_*, and AICDX_* columns
        **{col: 'max' for col in filtered_df.columns if col.startswith(('ATCX_', 'HEMI_', 'AICDX_'))},
    }
).reset_index()

In [ ]:
#RUN SAME

# Step 1: Filter features based on threshold
threshold = shap_threshold  # Adjust the threshold value as needed

if shap_threshold_level == "very_low": 
    threshold = threshold * 0.00000000000000000001
elif shap_threshold_level == "low":
    threshold = threshold * 0.001
elif shap_threshold_level == "medium":
    threshold = threshold * 0.01
elif shap_threshold_level == "high": 
    threshold = threshold * 0.1
    
important_features = original_feature_importance[original_feature_importance['Importance'] >= threshold]['Feature'].tolist()

# Step 2: Add any essential columns that should always be kept (e.g., PID, Subperiod)
essential_columns = ['PID', '1_Revision', 'Sex', 'Age']
columns_to_keep = essential_columns + important_features

# Step 3: Filter `result_df` to keep only the specified columns
filtered_df_DT = filtered_df_DT[columns_to_keep]
filtered_df_DT = filtered_df_DT.loc[:, ~filtered_df_DT.columns.duplicated()]
#filtered_df

In [ ]:
#DISCONTINUED SINCE WE ENDED UP RUNNING JUST WITH HIGH THRESHOLD
#CODE FOR THRESHOLD UPDATE (FOR THE TIME FRAME/RESOLUTION COMBINATION) AND LEVEL OF SEVERITY 
#RUN AFTER SHAP THRESHOLD OF SHAP THRESHOLD LEVEL HAVE BEEN UPDATED
importlib.reload(Config_V3)
shap_threshold = Config_V3.shap_threshold
shap_threshold_level = Config_V3.shap_threshold_level

In [ ]:
#I AM GETTING ONLY 80 FEATURES HAVE AN IMPORTANCE VALUE HIGER THAN 0 FROM ALL 2200. HIGHER THAN A .1 PER CENT LIKE LEANDRA
# ONLY 22 ATTRIBUTES, HIGHER THAN A 1 PER CENT ONLY 6 VARIABLES

In [ ]:
# Step 1: Filter for Subperiod == 1 to only use 1 entry data per PID
#filtered_df1 = filtered_df[filtered_df['Subperiod'] == 1]
filtered_df1 = filtered_df_DT

# Step 2: Drop unnecessary columns
filtered_df1 = filtered_df1.drop(columns=['PID'])

# Step 3: Prepare data for classification, division between predictors and target and between training and testing set
X = filtered_df1.drop(columns=['1_Revision'])
y = filtered_df1['1_Revision']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

# Step 4: Fit a classification tree model
model = DecisionTreeClassifier(random_state=0)
model.fit(X_train, y_train)

# Step 5: Make predictions on the test set
y_pred_proba = model.predict_proba(X_test)[:, 1]  # Probability estimates for the positive class
y_pred = model.predict(X_test)  # Binary predictions

# Step 6: Calculate ROC curve and AUC. fpr False Positive Rate, or, rate at which data points which should have been classified as negative, were as 
#positive, (false positives between all data points which should have been negative) tpr, same but for positives. AUC explanation below, as well as 
#threshold explanation
fpr, tpr, thresholds = roc_curve(y_test, y_pred_proba)
roc_auc = auc(fpr, tpr)

print(f"AUC: {roc_auc}")

# Plot ROC Curve. Explanation below
plt.figure()
plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--') #This is the diagonal line which represents random guessing, the worst possible performance
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
plt.legend(loc="lower right")
plt.show()

# Step 7: Confusion Matrix
# SubStep 1: Create the confusion matrix
cm = confusion_matrix(y_test, y_pred)

# SubStep 2: Create a DataFrame for better labeling
cm_df = pd.DataFrame(cm, index=['Actual Negative', 'Actual Positive'], 
                     columns=['Predicted Negative', 'Predicted Positive'])

# SubStep 3: Print the confusion matrix with labels
print("Confusion Matrix with labels:")
print(cm_df)

# SubStep 4: (Optional) Plot the confusion matrix for better visualization
plt.figure(figsize=(6,4))
sns.heatmap(cm_df, annot=True, fmt='g', cmap='Blues')
plt.title("Confusion Matrix")
plt.ylabel('Actual')
plt.xlabel('Predicted')
plt.show()

# Step 8: Youden's Index Calculation. Explanation for sensitivity, specificity and Youden Index down
sensitivity = tpr
specificity = 1 - fpr
youden_index = sensitivity + specificity - 1
optimal_idx = np.argmax(youden_index)
optimal_threshold = thresholds[optimal_idx] #Threshold value minimizing the Youden´s Index
optimal_youden = youden_index[optimal_idx]

print(f"Optimal Youden's Index: {optimal_youden}")
print(f"Optimal Threshold: {optimal_threshold}")

#Each dot represents an observation, blue dots are lower values of the feature and red higher ones. 
#A positive SHAP value means the feature pushes the prediction towards a higher output. 
#In the case of binary variables, points are red if they received the value of 1, and blue if they received a 0. SHAP values indicate for the particular 
#case where the medication or treatment was or was not assigned contribute to increasing or decreasing the prediction

# Step 10: SHAP Analysis
# Initialize the SHAP explainer
explainer = shap.TreeExplainer(model)

# Calculate SHAP values for each datapoint (How much features impact each prediction)
shap_values = explainer.shap_values(X_test)

#Shape of the shap_values array with the SHAP values for each feature. Then also the shape of the test set
print(f"SHAP values shape: {shap_values.shape}")
print(f"X_test shape: {X_test.shape}")

# Since we're doing binary classification, we usually focus on the positive class. It could be we are having a list containing SHAP values for the 
#0 class in the first place and class 1 in the second
    
if isinstance(shap_values, list):
    shap_values_class_1 = shap_values[1][:, :, 1]  # Focus only on the second class
else:
    shap_values_class_1 = shap_values[:, :, 1]  # If it's not a list, directly slice the second class
    
#The shape of the postive class array
print(f"SHAP values for class 1 shape: {shap_values_class_1.shape}")

#Average SHAP values for each feature across samples 
importance_values = np.abs(shap_values_class_1).mean(axis=0)  # Mean across the first dimension (samples)

#Check number of dimensions in the importance_values array. Each feature should have just 1 importance value. Take the mean if there are many, 
#averages importance values for a feature across samples
if importance_values.ndim > 1:
    importance_values = np.mean(importance_values, axis=1)

    #Debug knowing number of features and shape of the final importance array, they shall be same sized
print(f"Length of X.columns: {len(X.columns)}")
print(f"Shape of importance_values after mean calculation: {importance_values.shape}")


#Verifies the condition was actually satisfied and creates DataFrame feature/importance value
# Create a DataFrame for feature importance
if len(X.columns) == importance_values.shape[0]:
    feature_importance = pd.DataFrame({
        'Feature': X.columns,
        'Importance': importance_values
    })

# Sort feature importance
    feature_importance = feature_importance.sort_values(by='Importance', ascending=False)

    #Print the sorted feature importance based on the SHAP values. Or prints error if there is a mismatch between number of features and importance values
    pd.options.display.max_rows = None
# Display feature importance
    print("Feature Importance based on SHAP values:")
    print(feature_importance.head(1000))
else:
    print("Shape mismatch between features and importance values!")

# Plot SHAP summary plot for class 1
shap.summary_plot(shap_values_class_1, X_test)  # Plotting the first class SHAP values

In [ ]:
#RUN THIS, ONLY AFTER THE FIRST RUN WITHOUT ANY FEATURE SELECTION
original_feature_importance = feature_importance

## Bayesian Parameter Search

In [ ]:
# Define parameter space for Bayesian search
param_space = {
    'criterion': ['gini', 'entropy'],
    'max_depth': (Bay_max_depth_low, Bay_max_depth_high),
    'min_samples_split': (Bay_min_samples_split_low, Bay_min_samples_split_high),
    'min_samples_leaf': (Bay_min_samples_leaf_low, Bay_min_samples_leaf_high),
    'max_features': ['sqrt', 'log2', None]
}

# Initialize Bayesian Search
bayes_search = BayesSearchCV(estimator=DecisionTreeClassifier(random_state=0),
                             search_spaces=param_space,
                             scoring='roc_auc',
                             n_iter=Bay_n_iter,  # Number of iterations to run
                             cv=5,
                             verbose=1)

# Fit Bayesian search
bayes_search.fit(X_train, y_train)

# Best hyperparameters
print(f"Best parameters found: {bayes_search.best_params_}")

## Model Re fit making use of found parameters

###### ROC Curve: Evaluate the performance of a binary classification model. Plots the TPR (True Positive Rate or Recall). True Positives over True Positives + False Negatives, so, from all cases which should have been classified as Positive, how many of them where? and FPR (False Positive Rate)False Positives over False Positives + True Negatives, from all the ones which should have been classified as negative, how many where classifier as positive? The y axis is the TPR and the x axis is the FPR. Each point represents the trade off at different classification thresholds (Starting this value for the features the case will be classified as positive). A good curve will head to touch the upper left corner. 

In [ ]:
# Best parameters from Bayesian Search 
best_params = bayes_search.best_params_

# Step 1: Initialize the model with the best parameters
model = DecisionTreeClassifier(
    criterion=best_params['criterion'],
    max_depth=best_params['max_depth'],
    max_features=best_params['max_features'],
    min_samples_leaf=best_params['min_samples_leaf'],
    min_samples_split=best_params['min_samples_split'],
    random_state=0  # Keep random_state for reproducibility
)

# Step 2: Refit the model on the training data
model.fit(X_train, y_train)

# Step 3: Make predictions and evaluate the model
y_pred = model.predict(X_test)
y_proba = model.predict_proba(X_test)[:, 1]  # For AUC calculation

# Step 4: Calculate the AUC score
auc_score = roc_auc_score(y_test, y_proba)

print(f"Updated model AUC score: {auc_score:.4f}")

In [ ]:
print(filtered_df.columns)

###### AUC: Area under the ROC Curve, single scalar summarizing the performance of the model. 1 is perfect classifier and 0 is Worse than random guessing. Confusion Matrix: On x axis: Actual and on y axis: Predicted and so you have True Positive, False Negative, False Positive and True Negative. Finally, the Youden  Index takes into account the TPR and the TNR (From all the ones that should have been negative, how many were actually?). J = TPR + TNR -1. This metric is from 0 to 1, (if everything is perfectly classified, both have the value of 1, less 1 is 1. It can also be 0, here it is assumed that the worst possible result is random guessing, because actually, if you had a model which perfectly clasifies everything incorrectly, you could simple switch the value of the classification and it would result in a perfectmodel. Therefore, the worst possible value is 0, which occurs if both metrics get .5 for a -1 to total 0.) It provides the best tradeoff between True Positives and False Positives. 

## LSTM Modeling

In [ ]:
#Data expected in 3D format (samples, timesteps, features)

# Step 1: Sort by PID and Subperiod
#filtered_result_df2 = result_df.sort_values(by=['PID', 'Subperiod'])
filtered_df = filtered_df.loc[:, ~filtered_df.columns.duplicated()].copy()
filtered_df = filtered_df.sort_values(by=['PID', 'Subperiod'])
filtered_df = filtered_df[columns_to_keep]

# Step 2: Features to include for LSTM (excluding PID, Subperiod, PeriodStart, PeriodEnd)
features = []
optional_features = ['Age', 'Sex', 'Treatment_Gap_HEMI', 'Treatment_Gap_ATCX', 'Treatment_Gap_AICDX']  # Include ATCX, HEMI columns too
for col in optional_features:
    if col in filtered_df.columns:
        features.append(col)
        
atcx_cols = [col for col in filtered_df.columns if col.startswith('ATCX')]
hemi_cols = [col for col in filtered_df.columns if col.startswith('HEMI')]
icd_cols = [col for col in filtered_df.columns if col.startswith('AICDX')]
features = ['Age', 'Sex', 'Treatment_Gap_HEMI', 'Treatment_Gap_ATCX', 'Treatment_Gap_AICDX']
features = [col for col in features if col in filtered_df.columns] + atcx_cols + hemi_cols + icd_cols

if 'Age' in filtered_df.columns:
    filtered_df['Age'].fillna(filtered_df['Age'].mean(), inplace=True)
if 'Sex' in filtered_df.columns:
    sex_mode = filtered_df['Sex'].mode()
    if not sex_mode.empty:  # Check if there is a mode
        filtered_df['Sex'].fillna(sex_mode.iloc[0], inplace=True)  # Use .iloc[0] instead of [0]
    else:
        # Handle the case when there's no mode (optional, depends on your case)
        filtered_df['Sex'].fillna(filtered_df['Sex'].mode().mode[0], inplace=True)
        
        #filtered_df['Sex'].fillna(sex_mode[0], inplace=True)
#filtered_result_df2.fillna(value={'Age': filtered_result_df2['Age'].mean(), 'Sex': filtered_result_df2['Sex'].mode()[0]}, inplace=True)


# Step 3: Convert the dataframe into a 3D array (samples, timesteps, features)
X = []
y = []

if TimeResolution == "Quarters":  
    num_subperiods = ObservationDays/90
    
if TimeResolution == "Months":
    num_subperiods = ObservationDays/30
    
    

# Step 4: Group by PID and convert to sequences
for pid, group in filtered_df.groupby('PID'):
    
    #group = group[group.columns.intersection(filtered_df.columns)]
    
    if len(group) == num_subperiods:  # Ensure that all subperiods are present
        X.append(group[features].values)  # Get the features as 2D array for each patient
        y.append(group['1_Revision'].iloc[0])  # Revision surgery target (same for each subperiod)

X = np.nan_to_num(np.array(X))  # Shape: (num_patients, num_subperiods, num_features)
y = np.array(y)  # Shape: (num_patients,)

X[np.isnan(X)] = 0

# Step 5: Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

print("Shape of X_train:", X_train.shape)  # Should be (num_patients, num_subperiods, num_features)

In [ ]:
# Step 1: Fill NaNs i                                                                                                                                               n X with zeros before scaling
# Step 2: Flatten and scale the dataset, treating constant columns separately
# Reshape to (num_samples * num_timesteps, num_features) to scale the entire dataset
num_features = X_train.shape[2] #Features is in the third position and the count starts from 0
X_train_flat = X_train.reshape(-1, num_features) # Makes instead of separate number of patients and number of periods for each, a column like the origin
X_test_flat = X_test.reshape(-1, num_features)

# Find constant columns in X_train and X_test
constant_columns = [col for col in range(num_features) if np.ptp(X_train_flat[:, col]) == 0]
print("Constant columns:", constant_columns)


scaler = StandardScaler()
X_train_flat_scaled = X_train_flat.copy()
X_test_flat_scaled = X_test_flat.copy()

# Scale only columns that aren't constant
non_constant_columns = [col for col in range(num_features) if col not in constant_columns]
X_train_flat_scaled[:, non_constant_columns] = scaler.fit_transform(X_train_flat[:, non_constant_columns])
X_test_flat_scaled[:, non_constant_columns] = scaler.transform(X_test_flat[:, non_constant_columns])

# Reshape back to the original 3D shape
X_train_scaled = X_train_flat_scaled.reshape(X_train.shape)
X_test_scaled = X_test_flat_scaled.reshape(X_test.shape)

# Verify no NaNs in X_train_scaled or X_test_scaled
assert not np.isnan(X_train_scaled).any(), "NaNs found in X_train_scaled"
assert not np.isnan(X_test_scaled).any(), "NaNs found in X_test_scaled"

In [ ]:
#Step 1: Creates the model and establishes ranges of values for distint parameters like: number of units, adds dropout (unactivataing some neurons for
#overfitting avoiding purposes), adds the output layer with sigmoid activation, sets the range for the learning rate, compiles the model with 
#ADAM (Combination of accumulation of gradient for speeding up the process), names the loss as the Binary Cross Entropy
    
def build_model(hp):
    model = Sequential()
    
    # Tuning the number of LSTM units
    model.add(Conv1D(
        filters=hp.Int('filters', min_value=units_min_value, max_value=units_max_value, step=units_step),
        kernel_size=hp.Int('kernel_size', min_value=2, max_value=5),
        activation='relu',
        padding = 'same',
        input_shape=(X_train.shape[1], X_train.shape[2])
    ))
    
    
    model.add(Bidirectional(LSTM(
        units=hp.Int('units_0', min_value=units_min_value, max_value=units_max_value, step=units_step),
        return_sequences=True  # Add this line
    )))

    # Second LSTM layer with return_sequences=True to pass 3D data onward
    model.add(LSTM(
        units=hp.Int('units_1', min_value=units_min_value, max_value=units_max_value, step=units_step),
        return_sequences=True,
    ))
    
    # Third LSTM layer with return_sequences=False to output 2D data for Dense layer
    model.add(LSTM(
        units=hp.Int('units_2', min_value=units_min_value, max_value=units_max_value, step=units_step),
        return_sequences=False,
    ))

    # Tuning the dropout rate
    model.add(Dropout(rate=hp.Float('dropout', min_value=dropout_min_value, max_value=dropout_max_value, step=dropout_step)))
    
    # Output layer
    model.add(Dense(1, activation='sigmoid', kernel_regularizer=l2(0.01)))
    
    # Tuning the learning rate
    learning_rate = hp.Float('learning_rate', min_value=learning_rate_min_value, max_value=learning_rate_max_value, sampling='log')
    model.compile(optimizer=Adam(learning_rate=learning_rate),
                  loss='binary_crossentropy', metrics=['accuracy'])
    
    return model

In [ ]:
#Step 2: Sets the Bayesian Optimization procedure up: Sets the objective as increasing the validation accuracy, the max number of trials as 20
#HERE THERE IS A PARAMETER WHICH CAN BE EXTERNALIZED

def initialize_tuner():
    """Initializes the Bayesian Optimization tuner."""
    tuner = kt.BayesianOptimization(
        build_model,
        objective='val_accuracy',  # Or use 'val_loss' for minimizing validation loss
       max_trials=tuner_max_trials,
        directory='lstm_tuning',
        project_name='lstm_hyperparameter_tuning'
    )
    tuner.search_space_summary()
    return tuner

In [ ]:
#Step 3: Passes training and testing data, sets the size of the batches (subparts of the data which will be used for training and testing), stocks the 
#hyperparmeter values resulting in the best performance value

#HERE THERE IS ANOTHER PARAMETER WHICH CAN BE EXTERNALIZED
def perform_hyperparameter_search(tuner, X_train, y_train, X_test, y_test):
    """Performs hyperparameter tuning on the LSTM model."""
    tuner.search(X_train, y_train, epochs=20, batch_size=16, validation_data=(X_test, y_test))
    best_hps = tuner.get_best_hyperparameters(num_trials=5)[0]
    print(f"""
    The optimal number of units in the LSTM layer is {best_hps.get('units')}.
    The optimal dropout rate is {best_hps.get('dropout')}.
    The optimal learning rate is {best_hps.get('learning_rate')}.
    """)
    return best_hps

In [ ]:
#Step 4: Fits model with the best hyperparameters, calculates the accuracy and then calculates probabilities for the testing subset for revision
#Finally prints the AUC

#HERE THERE ARE MORE PARAMETERS WHICH CAN BE EXTERNALIZED

def train_and_evaluate(best_hps, X_train, y_train, X_test, y_test):
    """Builds, trains, and evaluates the model using the best hyperparameters."""
    best_model = build_model(best_hps)
    history = best_model.fit(X_train, y_train, epochs=num_epochs, batch_size= training_batch_size, validation_data=(X_test, y_test))
    loss, accuracy = best_model.evaluate(X_test, y_test)
    print(f"Test Accuracy: {accuracy:.4f}")

    # Generate predicted probabilities
    y_pred_prob = best_model.predict(X_test).flatten()
    y_pred = (y_pred_prob >= 0.5).astype(int) 
    
    # Calculate confusion matrix
    tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
    print(f"Confusion Matrix:\nTP: {tp}, FP: {fp}, TN: {tn}, FN: {fn}")

    # Calculate AUC
    auc = roc_auc_score(y_test, y_pred_prob)
    print(f"Test AUC: {auc:.4f}")

    return best_model, history  

In [ ]:
#Step 5: Accuracy and loss plots

def plot_accuracy(history):
    """Plots training and validation accuracy over epochs."""
    plt.figure(figsize=(12, 6))
    plt.plot(history.history['accuracy'], label='Train Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.title('Model Accuracy')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend(loc='lower right')
    plt.show()

def plot_loss(history):
    """Plots training and validation loss over epochs."""
    plt.figure(figsize=(12, 6))
    plt.plot(history.history['loss'], label='Train Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title('Model Loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(loc='upper right')
    plt.show()


In [ ]:
#Calls function by function: Calls the tuner, searches hyperparameters, trains and evaluates the best model found, plots accuracy and loss

def main(num_runs=8, seed_value=42):
    
    set_seed(seed_value)
    
    best_models = []
    best_histories = []
    
    
    for i in range(num_runs):
        print(f"\nRun {i + 1} of {num_runs}")
    # Initialize the tuner
        tuner = initialize_tuner()

    # Perform hyperparameter search
        best_hps = perform_hyperparameter_search(tuner, X_train_scaled, y_train, X_test_scaled, y_test)

    # Train and evaluate the model with best hyperparameters
        best_model, history = train_and_evaluate(best_hps, X_train_scaled, y_train, X_test_scaled, y_test)

    # Plot accuracy and loss
        best_models.append(best_model)
        best_histories.append(history)
        
        print(f"End of run {i + 1}")
        
    plot_accuracy(history)
    plot_loss(history)

# Run the main function if this file is executed
if __name__ == "__main__":
    main(num_runs=8)

In [ ]:
#We are seeing overfitting beahaviour: with training accuracy going up very high, training loss going very low, but testing accuracy actually going down
#and testing loss actually going up as epochs evolve

###### Difference between Accuracy and AUC: 

###### Accuracy is just total number of correct predictions over Total number of predictions (In a data set with 
###### example 995 positives and only 5 negatives, if you were predicting all positives the model would be 
###### 95 per cent right, but totally uncapable of identifying the negative cases)

###### AUC splits to look at how many of the ones which should have been positive, actually were, and how many 
###### of the ones which should have been negative, actually were. Obviously, with our kind of problem, AUC is 
###### preferred. 

### LSTM Reimplementation

In [ ]:
#New libraries

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
match TimeResolution:
        case "Months": 
            increment_days =  29
        case "Quarters": 
            increment_days = 89

#### Preprocessing

In [ ]:
# Preprocessing function
def preprocess_data(df, time_span_days=ObservationDays, time_resolution_days= increment_days + 1):
    # Calculate the number of subperiods for each patient
    df['SubperiodLength'] = (df['PeriodEnd'] - df['PeriodStart']).dt.days

    # Select relevant columns
    numeric_features = ['Age', 'Treatment_Gap_ATCX', 'Treatment_Gap_HEMI', 'Treatment_Gap_AICDX', 'Sex']
    binary_features = [col for col in df.columns if col.startswith(('ATCX', 'HEMI', 'AICDX'))]
    target_column = '1_Revision'

    # Impute missing values
    numeric_imputer = SimpleImputer(strategy='mean')
    df[numeric_features] = numeric_imputer.fit_transform(df[numeric_features])

    binary_imputer = SimpleImputer(strategy='constant', fill_value=0)
    df[binary_features] = binary_imputer.fit_transform(df[binary_features])

    # Normalize numeric features
    scaler = MinMaxScaler()
    df[numeric_features] = scaler.fit_transform(df[numeric_features])

    # Sort by PID and Subperiod
    df = df.sort_values(by=['PID', 'Subperiod']).reset_index(drop=True)

    # Group by PID to create sequences
    feature_columns = numeric_features + binary_features
    grouped = df.groupby('PID')

    sequences = []
    targets = []
    lengths = []

    for pid, group in grouped:
        # Ensure the group is within the time span
        valid_group = group[group['SubperiodLength'].cumsum() <= time_span_days]
        if not valid_group.empty:
            sequences.append(valid_group[feature_columns].values)
            targets.append(valid_group[target_column].iloc[-1])  # Use the last target value
            lengths.append(len(valid_group))

    # Pad sequences
    max_length = max(lengths)
    padded_sequences = pad_sequences(sequences, maxlen=max_length, dtype='float32', padding='post', truncating='post')

    return padded_sequences, np.array(targets), lengths, feature_columns



In [ ]:
def preprocess_data(df, time_span_days=ObservationDays, time_resolution_days= increment_days + 1):
    # Calculate the number of subperiods for each patient
    df['SubperiodLength'] = (df['PeriodEnd'] - df['PeriodStart']).dt.days

    # Define the selected features
    selected_features = [
    'AICDX_I48', 'ATCX_C01E', 'AICDX_F45', 'AICDX_K25', 'AICDX_R93',
    'AICDX_M93', 'AICDX_D04', 'ATCX_J01M', 'AICDX_T89', 'ATCX_N02A',
    'ATCX_B03A', 'AICDX_I70', 'AICDX_F43', 'ATCX_C08C', 'AICDX_M20',
    'AICDX_H19', 'AICDX_I71', 'AICDX_K57', 'AICDX_H35', 'AICDX_I49',
    'AICDX_Z96', 'ATCX_H02A', 'AICDX_H18', 'AICDX_H36', 'AICDX_I10',
    'AICDX_M84', 'AICDX_G47', 'AICDX_C80', 'AICDX_Q61',
    'ATCX_M01A', 'AICDX_M47', 'AICDX_M77', 'AICDX_J38', 'ATCX_C10A',
    'AICDX_F17', 'AICDX_E66', 'AICDX_R06', 'AICDX_C50', 'AICDX_R61',
    'AICDX_S82', 'ATCX_C09A', 'AICDX_M51', 'AICDX_M81', 'AICDX_F33',
    'AICDX_M50', 'AICDX_B99', 'HEMI_9901', 'ATCX_N06A', 'AICDX_R20',
    'AICDX_E79', 'AICDX_J06', 'AICDX_J98', 'AICDX_K80', 'HEMI_9701',
    'AICDX_G55', 'HEMI_1201', 'AICDX_J20', 'AICDX_I25', 'Sex',
    'AICDX_D64', 'AICDX_Z26', 'AICDX_F41', 'AICDX_E04', 'AICDX_I67',
    'AICDX_C85', 'AICDX_H25', 'HEMI_501', 'AICDX_U', 'AICDX_M65',
    'AICDX_I80', 'AICDX_Z25', 'AICDX_I73', 'AICDX_N28', 'AICDX_Z00',
    'ATCX_M03A', 'AICDX_H01', 'HEMI_106', 'HEMI_9933', 'ATCX_A03F',
    'AICDX_H43', 'AICDX_T88', 'AICDX_L03', 'AICDX_I65', 'AICDX_T81',
    'ATCX_D01A', 'AICDX_M46', 'AICDX_I38', 'ATCX_C01D', 'AICDX_R10',
    'AICDX_R26', 'ATCX_B01A', 'AICDX_M42', 'AICDX_R13', 'AICDX_Z24',
    'AICDX_R07', 'ATCX_D11A', 'HEMI_8030', 'AICDX_D48',
    'AICDX_R47', 'AICDX_Z90', 'AICDX_K40', 'AICDX_R12', 'AICDX_M23',
    'AICDX_R51', 'AICDX_A49', 'AICDX_R63', 'AICDX_Z98', 'Age',
    'ATCX_G03F', 'ATCX_M03B', 'ATCX_A02B', 'AICDX_F20', 'AICDX_UUU',
    'AICDX_N39', 'AICDX_Z13', 'AICDX_M62', 'AICDX_G62', 'AICDX_M16',
    'AICDX_Z12', 'ATCX_H03A', 'Treatment_Gap_ATCX', 'AICDX_J39', 'ATCX_J01C',
    'ATCX_C03C', 'AICDX_K21', 'ATCX_A06A', 'Treatment_Gap_HEMI', 'Treatment_Gap_AICDX',
    'AICDX_Z93', 'AICDX_Z97', 'AICDX_E11', 'AICDX_M48', 'AICDX_F32',
    'AICDX_M99', 'AICDX_Z27', 'ATCX_M05B', 'AICDX_I50', 'ATCX_D06A',
    'HEMI_201', 'AICDX_E34', 'AICDX_Z76', 'AICDX_H33', 'AICDX_S32',
    'AICDX_B00', 'AICDX_M19', 'ATCX_J04A', 'AICDX_K29', 'AICDX_M79',
    'AICDX_J32', 'AICDX_L89', 'AICDX_M25', 'AICDX_M54', 'AICDX_N18',
    'AICDX_Z47', 'AICDX_D29', 'ATCX_R03B', 'AICDX_R52', 'ATCX_C07A',
    'AICDX_H52', 'AICDX_E78', 'AICDX_H40', 'ATCX_P01A', 'AICDX_M53',
    'AICDX_T14', 'AICDX_Z01', 'AICDX_H91'
]

    # Select relevant columns
    binary_features = [col for col in df.columns if col.startswith(('ATCX', 'HEMI', 'AICDX'))]
    numeric_features = ['Age', 'Treatment_Gap_ATCX', 'Treatment_Gap_HEMI', 'Treatment_Gap_AICDX', 'Sex']
    selected_columns = selected_features  # Use only selected features for the model

    target_column = '1_Revision'

    # Impute missing values
    numeric_imputer = SimpleImputer(strategy='mean')
    df[numeric_features] = numeric_imputer.fit_transform(df[numeric_features])

    binary_imputer = SimpleImputer(strategy='constant', fill_value=0)
    df[binary_features] = binary_imputer.fit_transform(df[binary_features])

    # Normalize numeric features
    scaler = MinMaxScaler()
    df[numeric_features] = scaler.fit_transform(df[numeric_features])

    # Sort by PID and Subperiod
    df = df.sort_values(by=['PID', 'Subperiod']).reset_index(drop=True)

    # Group by PID to create sequences
    grouped = df.groupby('PID')

    sequences = []
    targets = []
    lengths = []

    for pid, group in grouped:
        # Ensure the group is within the time span
        valid_group = group[group['SubperiodLength'].cumsum() <= time_span_days]
        if not valid_group.empty:
            sequences.append(valid_group[selected_columns].values)
            targets.append(valid_group[target_column].iloc[-1])  # Use the last target value
            lengths.append(len(valid_group))

    # Pad sequences
    max_length = max(lengths)
    padded_sequences = pad_sequences(sequences, maxlen=max_length, dtype='float32', padding='post', truncating='post')

    return padded_sequences, np.array(targets), lengths, selected_columns


In [ ]:
# Example usage
padded_sequences, targets, lengths, feature_columns = preprocess_data(filtered_df)

### LSTM Implementation

In [ ]:
import torch
from torch.utils.data import DataLoader, Dataset
from torch import nn

# Define Dataset Class
class LSTMFeatureDataset(Dataset):
    def __init__(self, sequences, labels):
        self.sequences = sequences
        self.labels = labels

    def __len__(self):
        return len(self.sequences)
#Aquí vas tomando la info de paciente a paciente, tanto su secuencia de features, como su target
    def __getitem__(self, idx):
        return self.sequences[idx], self.labels[idx]

# Define LSTM Model
class LSTMModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers, output_dim, dropout=0.2):
        super(LSTMModel, self).__init__() #Initializes the parent class of LSTMModel, torch.nn.Module
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers = 1, batch_first=True, dropout=dropout)
        self.fc = nn.Linear(hidden_dim, output_dim) #Defines a fully connected layer for output creation

    def forward(self, x):
        lstm_out, _ = self.lstm(x)
        out = self.fc(lstm_out[:, -1, :])  # Use the last hidden state
        return out

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(padded_sequences, targets, test_size=0.2, random_state=42)

# Prepare Data for PyTorch
X_tensor = torch.tensor(X_train, dtype=torch.float32)
y_tensor = torch.tensor(y_train, dtype=torch.float32).unsqueeze(1)

# Convert to PyTorch tensors
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32).unsqueeze(1)

train_dataset = LSTMFeatureDataset(X_tensor, y_tensor)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

# Initialize Model, Loss, and Optimizer
input_dim = X_train.shape[2]
hidden_dim = 64
num_layers = 2
output_dim = 1
dropout = 0.7

model = LSTMModel(input_dim, hidden_dim, num_layers, output_dim, dropout)
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0005, weight_decay=.005)

# Train LSTM Model
num_epochs = 10
model.train()
for epoch in range(num_epochs):
    epoch_loss = 0
    for sequences, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(sequences)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
    print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {epoch_loss:.4f}")

# Evaluate Model on Train Set
model.eval()
with torch.no_grad():
    train_outputs = model(X_tensor)
    train_preds = torch.sigmoid(train_outputs).numpy()
    auc_score = roc_auc_score(y_train, train_preds)
    print(f"Train AUC: {auc_score:.4f}")

# Evaluate the model on the test set
model.eval()
with torch.no_grad():
    test_outputs = model(X_test_tensor)
    test_preds = torch.sigmoid(test_outputs).numpy()
    test_auc = roc_auc_score(y_test, test_preds)
    print(f"Test AUC: {test_auc:.4f}")

### Hyperparameter Tuning

In [ ]:
from hyperopt import hp, fmin, tpe, Trials

# Objective function for Bayesian Optimization
def objective(params):
    hidden_dim = int(params['hidden_dim'])  # Hyperparameter: number of units in the hidden layer
    num_layers = int(params['num_layers'])  # Hyperparameter: number of LSTM layers
    dropout = params['dropout']  # Hyperparameter: dropout rate
    lr = params['lr']  # Hyperparameter: learning rate
    
    model = LSTMModel(input_dim=X_train.shape[2], hidden_dim=hidden_dim, 
                      num_layers=num_layers, output_dim=1, dropout=dropout)
    
    criterion = nn.BCEWithLogitsLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    
    # Train the model
    model.train()
    num_epochs = 10
    for epoch in range(num_epochs):
        epoch_loss = 0
        for sequences, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(sequences)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()

    # Evaluate on train set
    model.eval()
    with torch.no_grad():
        train_outputs = model(X_tensor)
        train_preds = torch.sigmoid(train_outputs).numpy()
        auc_score = roc_auc_score(y_train, train_preds)
        
    return -auc_score  # Return negative AUC to minimize the objective function

In [ ]:
# Define the hyperparameter search space
space = {
    'hidden_dim': hp.quniform('hidden_dim', 16, 48, 16),  # Hidden dimension size
    'num_layers': hp.quniform('num_layers', 1, 2, 1),  # Number of LSTM layers
    'dropout': hp.uniform('dropout', 0.35, 0.7),  # Dropout rate
    'lr': hp.loguniform('lr', -10, -5.3),  # Learning rate (log-uniform)
    'weight_decay': hp.loguniform('weight_decay', -6, -3)  # Add weight regularization
}

In [ ]:
# Initialize Trials object to store optimization results
trials = Trials()

# Run the optimization
best = fmin(fn=objective, space=space, algo=tpe.suggest, max_evals=20, trials=trials)

# Output the best hyperparameters found
print("Best hyperparameters:", best)


In [ ]:
best_params = best
 
# Retrieve the best hyperparameters
hidden_dim_best = int(best_params['hidden_dim'])
num_layers_best = int(best_params['num_layers'])
dropout_best = best_params['dropout']
lr_best = best_params['lr']

# Retrain the model with the best hyperparameters
model = LSTMModel(input_dim=X_train.shape[2], hidden_dim=hidden_dim_best, 
                  num_layers=num_layers_best, output_dim=1, dropout=dropout_best)

optimizer = torch.optim.Adam(model.parameters(), lr=lr_best)

# Train the model again (you can use the same code as before)
# Train the model
model.train()
num_epochs = 10
for epoch in range(num_epochs):
    epoch_loss = 0
    for sequences, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(sequences)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()

# Evaluate and print the final AUC score on the test set
model.eval()
with torch.no_grad():
    test_outputs = model(X_test_tensor)
    test_preds = torch.sigmoid(test_outputs).numpy()
    test_auc = roc_auc_score(y_test, test_preds)
    print(f"Final Test AUC: {test_auc:.4f}")